<a href="https://colab.research.google.com/github/LeoDinga/DL_Project/blob/main/test_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y torch torchvision torchao torchaudio torchtext torchdata
!pip uninstall -y mmcv mmcv-full mmengine mmaction2 transformers

Found existing installation: protobuf 5.29.4
Uninstalling protobuf-5.29.4:
  Successfully uninstalled protobuf-5.29.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
grpcio-

In [ ]:
!pip install -U openmim
!mim install mmengine
!mim install 'mmcv==2.1.0'
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.35.2
!pip install mmaction2

In [ ]:
# verifica se tens o GPU ativado. Tem de dizer TRUE
# import numpy as np
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Version used by PyTorch: {torch.version.cuda}")

In [ ]:

# Exemplo para MMAction2
!git clone https://github.com/open-mmlab/mmaction2.git mmaction2 #alterei
%cd mmaction2
!pip install -e .

Cloning into 'dataset'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 113 (delta 0), reused 2 (delta 0), pack-reused 110 (from 1)
Receiving objects: 100% (113/113), 214.90 KiB | 5.51 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/dataset
remote: Enumerating objects: 1972, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1972 (delta 0), reused 0 (delta 0), pack-reused 1971 (from 1)
Receiving objects: 100% (1972/1972), 3.63 GiB | 34.39 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (1981/1981), done.
Your branch is up to date with 'origin/main'.


In [ ]:
!mim install 'mmcv==2.1.0'

In [ ]:
import mmcv
# import mmengine #not sure
import mmaction2

In [ ]:
import sys
sys.path.append('mmaction2')

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')


save_path = '/content/drive/MyDrive/mmaction2_work_dir/stgcn_treino'
os.makedirs(save_path, exist_ok=True)

In [ ]:

config_code = """

_base_ = ['/content/mmaction2/configs/_base_/default_runtime.py']
model = dict(
    type='RecognizerGCN',
    backbone=dict(
        type='STGCN',
        graph_cfg=dict(layout='nturgb+d', mode='stgcn_spatial'),
        in_channels=3
    ),
    cls_head=dict(type='GCNHead', num_classes=12, in_channels=256)
)

train_pipeline = [
    dict(type='PreNormalize3D'),
    dict(type='GenSkeFeat', dataset='nturgb+d', feats=['j']),
    dict(type='UniformSampleFrames', clip_len=100),
    dict(type='PoseDecode'),
    dict(type='FormatGCNInput', num_person=1),
    dict(type='PackActionInputs')
]
val_pipeline = [
    dict(type='PreNormalize3D'),
    dict(type='GenSkeFeat', dataset='nturgb+d', feats=['j']),
    dict(type='UniformSampleFrames', clip_len=100, num_clips=1, test_mode=True),
    dict(type='PoseDecode'),
    dict(type='FormatGCNInput', num_person=1),
    dict(type='PackActionInputs')
]
test_pipeline = [
    dict(type='PreNormalize3D'),
    dict(type='GenSkeFeat', dataset='nturgb+d', feats=['j']),
    dict(type='UniformSampleFrames', clip_len=100, num_clips=1, test_mode=True),
    dict(type='PoseDecode'),
    dict(type='FormatGCNInput', num_person=1),
    dict(type='PackActionInputs')
]

train_dataloader = dict(
    batch_size=64,  # ALTERADO: era 16, agora 4 para teste rápido
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset=dict(
        type='PoseDataset',
        ann_file='/content/train_labels.pkl',  # ALTERADO: caminho absoluto, antes era 'train_labels.pkl'
        pipeline=train_pipeline
    )
  )

val_dataloader = dict(
    batch_size=64,  # ALTERADO: era 16, agora 4 para teste rápido
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='PoseDataset',
        ann_file='/content/val_labels.pkl',  # ALTERADO: caminho absoluto, antes era 'val_labels.pkl'
        pipeline=val_pipeline
    )
)
test_dataloader = dict(
    batch_size=64,  # ALTERADO: era 16, agora 4 para teste rápido
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='PoseDataset',
        ann_file='/content/test_labels.pkl',  # ALTERADO: caminho absoluto, antes era 'test_labels.pkl'
        pipeline=test_pipeline
    )
)



param_scheduler = [
    dict(
        type='CosineAnnealingLR',
        eta_min=0,
        T_max=16,
        by_epoch=True,
        convert_to_iter_based=True)
]

evaluation = dict(
    interval=2,
    metrics=['top_k_accuracy']
)

val_evaluator = [dict(type='AccMetric')]
test_evaluator = val_evaluator



train_cfg = dict(
    type='EpochBasedTrainLoop', max_epochs=50, val_begin=1, val_interval=1)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')


optim_wrapper = dict(
    optimizer=dict(
        type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0005, nesterov=True)
)


# Salvar checkpoints, logs, etc.
work_dir = '/content/drive/MyDrive/mmaction2_work_dir/stgcn_treino'   # ALTERADO: era './work_dirs/stgcn_3d_custom_test'

load_from = 'https://download.openmmlab.com/mmaction/v1.0/skeleton/stgcn/stgcn_8xb16-joint-u100-80e_ntu60-xsub-keypoint-3d/stgcn_8xb16-joint-u100-80e_ntu60-xsub-keypoint-3d_20221129-850308e1.pth'



"""

# Gravar para o ficheiro na pasta configs/stgcn/
with open('/content/mmaction2/configs/stgcn_3d_custom.py', 'w') as f:
    f.write(config_code)

In [ ]:

config_code = """

_base_ = ['/content/mmaction2/configs/_base_/default_runtime.py']
model = dict(
    type='RecognizerGCN',
    backbone=dict(
        type='STGCN',
        graph_cfg=dict(layout='nturgb+d', mode='stgcn_spatial'),
        in_channels=3
    ),
    cls_head=dict(type='GCNHead', num_classes=12, in_channels=256)
)

train_pipeline = [
    dict(type='PreNormalize3D'),
    dict(type='GenSkeFeat', dataset='nturgb+d', feats=['j']),
    dict(type='UniformSampleFrames', clip_len=100),
    dict(type='PoseDecode'),
    dict(type='FormatGCNInput', num_person=1),
    dict(type='PackActionInputs')
]
val_pipeline = [
    dict(type='PreNormalize3D'),
    dict(type='GenSkeFeat', dataset='nturgb+d', feats=['j']),
    dict(type='UniformSampleFrames', clip_len=100, num_clips=1, test_mode=True),
    dict(type='PoseDecode'),
    dict(type='FormatGCNInput', num_person=1),
    dict(type='PackActionInputs')
]
test_pipeline = [
    dict(type='PreNormalize3D'),
    dict(type='GenSkeFeat', dataset='nturgb+d', feats=['j']),
    dict(type='UniformSampleFrames', clip_len=100, num_clips=1, test_mode=True),
    dict(type='PoseDecode'),
    dict(type='FormatGCNInput', num_person=1),
    dict(type='PackActionInputs')
]

train_dataloader = dict(
    batch_size=64,  # ALTERADO: era 16, agora 4 para teste rápido
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset=dict(
        type='PoseDataset',
        ann_file='/content/train_labels.pkl',  # ALTERADO: caminho absoluto, antes era 'train_labels.pkl'
        pipeline=train_pipeline
    )
  )

val_dataloader = dict(
    batch_size=64,  # ALTERADO: era 16, agora 4 para teste rápido
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='PoseDataset',
        ann_file='/content/val_labels.pkl',  # ALTERADO: caminho absoluto, antes era 'val_labels.pkl'
        pipeline=val_pipeline
    )
)
test_dataloader = dict(
    batch_size=64,  # ALTERADO: era 16, agora 4 para teste rápido
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
      type='PoseDataset',
      ann_file='/content/our_test_labels.txt',  # Pode ser .txt MUDEI
      pipeline=test_pipeline
)

)



param_scheduler = [
    dict(
        type='CosineAnnealingLR',
        eta_min=0,
        T_max=16,
        by_epoch=True,
        convert_to_iter_based=True)
]

evaluation = dict(
    interval=2,
    metrics=['top_k_accuracy']
)

val_evaluator = [dict(type='AccMetric')]
test_evaluator = val_evaluator



train_cfg = dict(
    type='EpochBasedTrainLoop', max_epochs=50, val_begin=1, val_interval=1)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')


optim_wrapper = dict(
    optimizer=dict(
        type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0005, nesterov=True)
)


# Salvar checkpoints, logs, etc.
work_dir = '/content/drive/MyDrive/mmaction2_work_dir/stgcn_treino'   # ALTERADO: era './work_dirs/stgcn_3d_custom_test'

load_from = 'https://download.openmmlab.com/mmaction/v1.0/skeleton/stgcn/stgcn_8xb16-joint-u100-80e_ntu60-xsub-keypoint-3d/stgcn_8xb16-joint-u100-80e_ntu60-xsub-keypoint-3d_20221129-850308e1.pth'



"""

# Gravar para o ficheiro na pasta configs/stgcn/
with open('/content/mmaction2/configs/stgcn_3d_custom.py', 'w') as f:
    f.write(config_code)

In [ ]:
!python tools/test.py configs/stgcn_3d_custom.py /content/drive/MyDrive/mmaction2_work_dir/stgcn_treino/epoch_50.pth --dump /content/drive/MyDrive/mmaction2_work_dir/stgcn_test_50/results.pkl
